<a href="https://colab.research.google.com/github/kbw612/Fantasy/blob/main/colab/DraftKingsWeeklyFantasyPoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def clean_player_name(player):
  end_index = player.index(' (')
  if end_index != -1:
    return player[0:end_index]

  return player

def get_player_touchdowns_df(week):
  receiving_stats_df = pd.read_csv(f'https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/stats/4for4-Player-Receiving-Stats.csv?1-2')
  receiving_stats_df = receiving_stats_df[receiving_stats_df["week"] == week]
  receiving_stats_df = receiving_stats_df[['full_name', 'posteam', 'position', 'rec_tds']]

  rushing_stats_df = pd.read_csv(f'https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/stats/4for4-Player-Rushing-Stats.csv?1=2')
  rushing_stats_df = rushing_stats_df[rushing_stats_df["week"] == week]
  rushing_stats_df = rushing_stats_df[['full_name', 'posteam', 'position', 'rush_tds']]

  #passing_stats_df = pd.read_csv(f'https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/stats/4for4-Player-Passing-Stats.csv')
  #passing_stats_df = passing_stats_df[passing_stats_df["week"] == week]
  #passing_stats_df = passing_stats_df[['full_name', 'posteam', 'position', 'rush_tds']]

  tds_df = pd.merge(rushing_stats_df, receiving_stats_df, left_on="full_name", right_on="full_name", how="outer", suffixes=('_1', '_2'))
  tds_df['team'] = tds_df['posteam_1'].combine_first(tds_df['posteam_2'])
  tds_df['position'] = tds_df['position_1'].combine_first(tds_df['position_2'])
  tds_df = tds_df[['full_name', 'team', 'position', 'rush_tds', 'rec_tds']]
  tds_df['rush_tds'] = tds_df['rush_tds'].fillna(0)
  tds_df['rec_tds'] = tds_df['rec_tds'].fillna(0)

  tds_df['TD_Points'] = (tds_df['rush_tds'] * 6) + (tds_df['rec_tds'] * 6)
  tds_df = tds_df[['full_name', 'TD_Points']].rename(columns={'full_name': 'Player'})

  return tds_df

current_week = 18
previous_week = current_week - 1
position = 'WR'

td_players_df = get_player_touchdowns_df(previous_week)
#print(tds_df)

current_week_salary_df = pd.read_csv(f'https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DKSalaries-week{current_week}.csv?1=3')
current_week_salary_df = current_week_salary_df.drop(columns=['Position', 'Name + ID', 'ID', 'Roster Position', 'Game Info', 'TeamAbbrev', 'AvgPointsPerGame'], axis=1)
for row in current_week_salary_df.index:
  current_week_salary_df.at[row, "Name"] = current_week_salary_df.at[row, "Name"].strip()

#previous_week_ownership_df = pd.read_csv("https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/dk-week2-contest-standings-150236571-ownership-percent.csv")

previous_week_ownership_and_points_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/dk-week{previous_week}-contest-standings-mon-thur-all-players-fantasy-points.csv?1=2")
previous_week_ownership_and_points_df = previous_week_ownership_and_points_df.drop(columns=['Rank', 'EntryId', 'EntryName', 'TimeRemaining', 'Points', 'Lineup', 'Roster Position', 'Unnamed: 6'], axis=1)
previous_week_ownership_and_points_df["Player"] = previous_week_ownership_and_points_df["Player"].str.strip()

players_df = pd.read_csv("https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DK-Players.csv")
#players_df = players_df.drop(columns=['Week16_%Drafted', 'Week16_FPTS', 'Week16_Non_TD_FPTS', 'Week17_Salary'])

players_df = pd.merge(players_df, previous_week_ownership_and_points_df, left_on='Name', right_on='Player', how='left').drop(columns=['Player'], axis=1).rename(columns={'FPTS': f'Week{previous_week}_FPTS', '%Drafted': f'Week{previous_week}_%Drafted', 'Position_x': 'Position'})
players_df = pd.merge(players_df, td_players_df, left_on='Name', right_on='Player', how='left').drop(columns=['Player'], axis=1).rename(columns={'TD_Points': f'Week{previous_week}_TD_Points'})
players_df[f'Week{previous_week}_Non_TD_FPTS'] = players_df[f'Week{previous_week}_FPTS'] - players_df[f'Week{previous_week}_TD_Points']
players_df = players_df.drop(columns=[f'Week{previous_week}_TD_Points'], axis=1)
players_df = pd.merge(players_df, current_week_salary_df, left_on='Name', right_on='Name', how='left').rename(columns={'Salary': f'Week{current_week}_Salary'})

players_df.to_csv(f'DK-Players-week-{current_week}.csv', index=False)
